In [4]:

"""
Banknote Authentication from Images - Real vs Fake Classifier
User uploads an image and the system predicts if it's real or fake
"""

# Install required packages
!pip install gradio scikit-learn pandas numpy matplotlib seaborn opencv-python pillow --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
import cv2
from PIL import Image
import io
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set random seed for consistent results
np.random.seed(42)

print("All packages installed and imported successfully")

# Step 1: Load and prepare the training data
print("Loading banknote data for training...")

data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
column_names = ['variance', 'skewness', 'kurtosis', 'entropy', 'class']
banknote_data = pd.read_csv(data_url, header=None, names=column_names)

print(f"Dataset loaded with {banknote_data.shape[0]} banknotes")

# Prepare features and target
X = banknote_data.drop('class', axis=1)
y = banknote_data['class']

print("First 5 rows of the data:")
print(banknote_data.head())

# Step 2: Train machine learning models
print("\nTraining machine learning models...")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train models
models = {}

# Decision Tree
dt_model = DecisionTreeClassifier(random_state=42, max_depth=5)
dt_model.fit(X_train_scaled, y_train)
models['Decision Tree'] = dt_model

# Random Forest
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train_scaled, y_train)
models['Random Forest'] = rf_model

# Support Vector Machine
svm_model = SVC(random_state=42, probability=True)
svm_model.fit(X_train_scaled, y_train)
models['SVM'] = svm_model

print("All models trained successfully")

# Evaluate models to choose the best one
results = []
for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    results.append({'Model': name, 'Accuracy': accuracy})

results_df = pd.DataFrame(results)
best_model_name = results_df.loc[results_df['Accuracy'].idxmax(), 'Model']
best_model = models[best_model_name]

print(f"Best model: {best_model_name} with accuracy: {results_df['Accuracy'].max():.4f}")

# Save the best model and scaler
joblib.dump(best_model, 'banknote_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

print("Model and scaler saved successfully")

# Step 3: Create function to extract features from images
def extract_features_from_image(image):
    """
    Extract variance, skewness, kurtosis, and entropy from an image
    """
    # Convert image to numpy array if it's a PIL Image
    if isinstance(image, Image.Image):
        image = np.array(image)

    # Convert to grayscale if it's a color image
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Resize image to standard size for consistency
    image = cv2.resize(image, (200, 100))

    # Flatten the image to 1D array
    pixels = image.flatten().astype(float)

    # Calculate features
    variance = np.var(pixels)
    skewness = pd.Series(pixels).skew()
    kurtosis = pd.Series(pixels).kurtosis()

    # Calculate entropy
    histogram = np.histogram(pixels, bins=256, range=(0, 255))[0]
    histogram = histogram / histogram.sum()
    entropy = -np.sum(histogram * np.log2(histogram + 1e-10))  # Add small value to avoid log(0)

    return variance, skewness, kurtosis, entropy

# Step 4: Create prediction function for uploaded images
def predict_banknote_from_image(image):
    """
    Predict whether an uploaded banknote image is real or fake
    """
    try:
        # Extract features from the image
        variance, skewness, kurtosis, entropy = extract_features_from_image(image)

        # Create feature array
        features = np.array([[variance, skewness, kurtosis, entropy]])

        # Load the scaler and model
        scaler = joblib.load('scaler.pkl')
        model = joblib.load('banknote_model.pkl')

        # Scale the features
        features_scaled = scaler.transform(features)

        # Make prediction
        prediction = model.predict(features_scaled)[0]
        probability = model.predict_proba(features_scaled)[0]

        # Prepare results
        if prediction == 0:
            result = "REAL Banknote"
            confidence = probability[0] * 100
            color = "green"
        else:
            result = "FAKE Banknote"
            confidence = probability[1] * 100
            color = "red"

        # Create detailed output
        output = f"""
        PREDICTION RESULT:
        -----------------
        Result: {result}
        Confidence: {confidence:.2f}%

        EXTRACTED FEATURES:
        ------------------
        Variance: {variance:.4f}
        Skewness: {skewness:.4f}
        Kurtosis: {kurtosis:.4f}
        Entropy: {entropy:.4f}

        NOTE: These features were automatically calculated from your image.
        """

        return output

    except Exception as e:
        return f"Error processing image: {str(e)}"


# Step 6: Create web interface
print("\nCreating web interface...")

# Create the interface
def create_interface():
    with gr.Blocks(title="Banknote Authenticity Checker") as demo:
        gr.Markdown("# Banknote Authenticity Checker")


        with gr.Row():
            with gr.Column():
                image_input = gr.Image(
                    label="Upload Banknote Image",
                    type="numpy",
                    height=300
                )
                submit_btn = gr.Button("Analyze Banknote", variant="primary")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Analysis Results",
                    lines=15,
                    max_lines=20
                )


        # Function to handle submission
        submit_btn.click(
            fn=predict_banknote_from_image,
            inputs=image_input,
            outputs=output_text
        )

        # Footer
        gr.Markdown("""
        ---
        **Note**: This is a demonstration system. For real banknote verification,
        always use official methods and tools provided by financial institutions.
        """)

    return demo

# Create and launch the interface
print("Launching web application...")
demo = create_interface()
demo.launch(share=True, debug=False)



All packages installed and imported successfully
Loading banknote data for training...
Dataset loaded with 1372 banknotes
First 5 rows of the data:
   variance  skewness  kurtosis  entropy  class
0   3.62160    8.6661   -2.8073 -0.44699      0
1   4.54590    8.1674   -2.4586 -1.46210      0
2   3.86600   -2.6383    1.9242  0.10645      0
3   3.45660    9.5228   -4.0112 -3.59440      0
4   0.32924   -4.4552    4.5718 -0.98880      0

Training machine learning models...
All models trained successfully
Best model: SVM with accuracy: 1.0000
Model and scaler saved successfully

Creating web interface...
Launching web application...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dbb72e8a12bade5841.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
